In [4]:
import numpy as np

a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])

np.add(a, b)

array([11, 22, 33, 44])

In [5]:
np.add(a, 100)

array([101, 102, 103, 104])

In [6]:
c = np.arange(4*4).reshape((4,4))
print('c:', c)

np.add(b, c)

c: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


array([[10, 21, 32, 43],
       [14, 25, 36, 47],
       [18, 29, 40, 51],
       [22, 33, 44, 55]])

In [7]:
b_col = b[:, np.newaxis]
b_col

array([[10],
       [20],
       [30],
       [40]])

In [8]:
np.add(b_col, c)

array([[10, 11, 12, 13],
       [24, 25, 26, 27],
       [38, 39, 40, 41],
       [52, 53, 54, 55]])

In [9]:
from numba import vectorize

@vectorize(['int64(int64, int64)'], target='cuda')
def add_ufunc(x, y):
    return x + y

In [10]:
%timeit np.add(b_col, c)   # NumPy on CPU

628 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
import math  # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.  Numba will inline it at compile time.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [12]:
# Evaluate the Gaussian a million times!
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test
gaussian_pdf(x[0], 0.0, 1.0)

C:\Users\alfal\anaconda3\envs\AI\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([0.12681967], dtype=float32)

In [13]:
gaussian_pdf(x[0], 0.0, 37.0)

array([0.0107732], dtype=float32)

In [14]:
import scipy.stats # for definition of gaussian distribution
norm_pdf = scipy.stats.norm
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

21.7 ms ± 623 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numpy as np
import cupy as cp
from numba import njit

@njit
def solve_roessler(x, y, z, dx, dy, dt, a, b, c):
    dx_new = (-y - z) * dt
    dy_new = (x + a * y) * dt
    dz_new = (b + z * (x - c)) * dt

    x += dx_new
    y += dy_new
    z += dz_new

    return x, y, z, dx_new, dy_new

# Set the initial conditions on CPU
x, y, z = 10, 10, 10
dx, dy = 0.0, 0.0
dt = 0.0001
a, b, c = 0.1, 0.1, 14.0
scale = 10

# Transfer initial conditions to GPU
x_gpu, y_gpu, z_gpu = cp.array(x), cp.array(y), cp.array(z)
dx_gpu, dy_gpu = cp.array(dx), cp.array(dy)

# Main simulation loop
while True:
    x_gpu, y_gpu, z_gpu, dx_gpu, dy_gpu = solve_roessler(x_gpu, y_gpu, z_gpu, dx_gpu, dy_gpu, dt, a, b, c)

    # The rest of the code for turtle graphics remains on the CPU
    t.setpos(x_gpu * scale, y_gpu * scale)
    t.setheading(atan2(dy_gpu, dx_gpu))
    t.pencolor(cp.asnumpy(cp.random.rand(3)))


In [19]:
import numpy as np
from numba import cuda

@cuda.jit
def rossler_attractor_kernel(x, y, z, dx, dy, a, b, c, dt, scale, iterations):
    i = cuda.grid(1)
    if i < iterations:
        dx[i] = (-y[i] - z[i]) * dt
        dy[i] = (x[i] + a * y[i]) * dt
        dz = (b + z[i] * (x[i] - c)) * dt
        x[i] += dx[i]
        y[i] += dy[i]
        z[i] += dz

def rossler_attractor(a, b, c, dt, scale, iterations):
    x = np.full(iterations, 10.0)
    y = np.full(iterations, 10.0)
    z = np.full(iterations, 10.0)
    dx = np.zeros(iterations)
    dy = np.zeros(iterations)

    threads_per_block = 256
    blocks_per_grid = (iterations + (threads_per_block - 1)) // threads_per_block

    rossler_attractor_kernel[blocks_per_grid, threads_per_block](
        x, y, z, dx, dy, a, b, c, dt, scale, iterations
    )

    return x, y, z

# Set the parameters
a = 0.1
b = 0.1
c = 14.0
dt = 0.0001  # time step
scale = 10
iterations = 1000000  # number of iterations

# Run the simulation
x, y, z = rossler_attractor(a, b, c, dt, scale, iterations)


C:\Users\alfal\anaconda3\envs\AI\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
import turtle
import numpy as np
from math import atan2
from numba import jit

# Create a turtle and set the pen size
turtle.screensize(canvwidth=7680, canvheight=4800, bg='black')
t = turtle.Turtle()
t.speed('fastest')
t.pensize(1)
t.pencolor(np.random.rand(3))
t.radians()
t.pendown()

@jit(nopython=True)
def calculate_trajectory(x, y, z, dx, dy, a, b, c, dt, scale, iterations):
    trajectory = np.zeros((iterations, 2))
    for i in range(iterations):
        trajectory[i][0] = x * scale
        trajectory[i][1] = y * scale

        dx = (-y - z) * dt
        dy = (x + a * y) * dt
        dz = (b + z * (x - c)) * dt

        x += dx
        y += dy
        z += dz
    return trajectory

# Set the parameters
a = 0.1
b = 0.1
c = 14.0
dt = 0.0001  # time step
scale = 10
iterations = 100000  # number of iterations

# Set the initial conditions
x, y, z = 10, 10, 10
dx, dy = 0.0, 0.0  # initialize the x velocity

trajectory = calculate_trajectory(x, y, z, dx, dy, a, b, c, dt, scale, iterations)



# Plot the trajectory
for pos in trajectory:
    t.setpos(pos[0], pos[1])
    # Set the pen color to a random color
    

# Keep the window open until it is closed
turtle.mainloop()
